# Aula 3: Uso de Modelos de Visão Computacional em Pipelines Reais

**Objetivo:** Implementar um pipeline completo de visão computacional usando um modelo YOLOv8 pré-treinado para detectar, classificar e contar objetos em um vídeo.

> **Conceitos-chave abordados**:
> - **Pipeline de Visão**: Um fluxo de trabalho automatizado que integra pré-processamento, inferência de modelo e pós-processamento.
> - **Inferência**: O processo de usar um modelo treinado para fazer previsões em novos dados (neste caso, quadros de vídeo).
> - **Rastreamento de Objetos**: Manter um ID único para cada objeto detectado ao longo de vários quadros para permitir uma contagem precisa.

**Referências**

- [Documentação da Ultralytics (YOLOv8)](https://docs.ultralytics.com/)
- [Repositório do Roboflow Supervision](https://github.com/roboflow/supervision)


**Dica💡**

A demonstração prática desta aula ajuda bastante neste exercício.

## 📋 Fluxo do exercício

1.  **Configurar o ambiente** instalando as bibliotecas necessárias (`ultralytics`, `supervision`).
2. **Complete os trechos de código entre**
```
### =============================================
### --------> Inicie seu código aqui <-----------
### =============================================
```
e
```
### =============================================
### ----------> Fim do seu código <--------------
### =============================================
```
3. Cada seção possui **sanity checks** para verificar se a implementação está correta.
4. Você pode adicionar células extras para **experimentação**.
5.  **Testar e validar** o contador de pessoas com um novo vídeo ou com a webcam.

> 🔧 **Linhas que você deverá modificar** na Tarefa Individual estão marcadas com:  
> `### INICIE SEU CÓDIGO AQUI ###` e `### FIM DO SEU CÓDIGO ###`

## ⚙️ 1. Ambiente e Dependências

Execute a célula abaixo para instalar as bibliotecas `ultralytics` (para o modelo YOLOv8) e `supervision` (para utilitários de rastreamento e anotação). Se estiver no Colab, o ambiente já vem com `opencv-python` e `numpy`.

In [1]:
# Instala as dependências necessárias
!pip install -q ultralytics supervision

# Importa as bibliotecas
import cv2
import supervision as sv
from ultralytics import YOLO
import numpy as np
import os
import urllib.request
from collections import defaultdict

# Criação de alguns dados de teste utilizados nos sanity checks
mock_xyxy = np.array([[10, 10, 20, 20], [30, 30, 40, 40], [50, 50, 60, 60]])
mock_confidence = np.array([0.9, 0.8, 0.7])
mock_class_id = np.array([0, 2, 0])
mock_detections = sv.Detections(xyxy=mock_xyxy, confidence=mock_confidence, class_id=mock_class_id)

print("Dependências instaladas e importadas com sucesso!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 19.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dependências instaladas e importadas com sucesso!


## 👤 2. Contador de Pessoas em Tempo Real

O objetivo é adaptar o código anterior para criar um pipeline que conte o número de pessoas em cada quadro de um vídeo.

**Sua tarefa:**
1.  **Encontrar um vídeo** com pessoas ou **habilitar a webcam**.
2.  **Alterar o filtro de classes** para detectar apenas a classe "person". O ID da classe "person" no dataset COCO (usado pelo YOLOv8) é **0**.
3.  **Implementar uma lógica de contagem** que exiba o número de pessoas detectadas no quadro atual.
4.  **Anotar o quadro** com o texto da contagem.

Use a célula de código abaixo, que já contém a estrutura da demonstração, e preencha as partes marcadas.

In [2]:
OUTPUT_VIDEO_PATH = "output_video.mp4"

### =============================================
### --------> Inicie seu código aqui <-----------
### =============================================

# Nome do arquivo que será baixado
VIDEO_PATH = "zap.mp4"

if not os.path.exists(VIDEO_PATH):
    print("Baixando o vídeo do GitHub...")
    !wget -q -O "{VIDEO_PATH}" "{VIDEO_URL}"
    print("Download completo!")

### =============================================
### -----------> Fim do seu código <-------------
### =============================================

assert os.path.exists(VIDEO_PATH), f"❌ Erro: Vídeo não encontrado em '{VIDEO_PATH}'"
print("✅ 1/2: Arquivo do vídeo existe.")

file_size = os.path.getsize(VIDEO_PATH)
assert file_size > 1000000, f"❌ Erro: O arquivo de vídeo é menor que 1MB ({file_size / 1e6:.2f} MB). O download pode ter falhado."
print(f"✅ 2/2: Arquivo do vídeo não está vazio (Tamanho: {file_size / 1e6:.2f} MB).")

print("✅ Sanity check aprovado!")

✅ 1/2: Arquivo do vídeo existe.
✅ 2/2: Arquivo do vídeo não está vazio (Tamanho: 2.50 MB).
✅ Sanity check aprovado!


In [5]:
OUTPUT_VIDEO_PATH = "output_video.mp4"

### =============================================
### --------> Inicie seu código aqui <-----------
### =============================================

# Obtém informações do vídeo
video_info = sv.VideoInfo.from_video_path(video_path="zap.mp4")

# Cria instâncias dos anotadores (versão corrigida)
box_annotator = sv.BoxAnnotator(thickness=4)
label_annotator = sv.LabelAnnotator(text_thickness=4, text_scale=2)

# Carrega o modelo (Sugestão : yolov8n.pt)
model = YOLO("yolov8n.pt")

### =============================================
### -----------> Fim do seu código <-------------
### =============================================

assert isinstance(video_info, sv.VideoInfo), f"❌ Erro: 'video_info' não é um objeto do tipo supervision.VideoInfo."
print(f"✅ 1/4: Video Info criado com sucesso (Resolução: {video_info.width}x{video_info.height}).")

assert isinstance(box_annotator, sv.BoxAnnotator), f"❌ Erro: 'box_annotator' não é um objeto do tipo supervision.BoxAnnotator."
print("✅ 2/4: Objeto 'box_annotator' criado com sucesso.")

assert isinstance(label_annotator, sv.LabelAnnotator), f"❌ Erro: 'label_annotator' não é um objeto do tipo supervision.LabelAnnotator."
print("✅ 3/4: Objeto 'label_annotator' criado com sucesso.")

assert hasattr(model, 'names'), f"❌ Erro: Modelo YOLO carregado incorretamente"
print("✅ 4/4: Modelo carregado com sucesso.")

print("✅ Sanity check aprovado!")

✅ 1/4: Video Info criado com sucesso (Resolução: 464x832).
✅ 2/4: Objeto 'box_annotator' criado com sucesso.
✅ 3/4: Objeto 'label_annotator' criado com sucesso.
✅ 4/4: Modelo carregado com sucesso.
✅ Sanity check aprovado!


In [6]:
def get_detections_from_frame(frame) -> sv.Detections:
    ### =============================================
    ### --------> Inicie seu código aqui <-----------
    ### =============================================

    # Executa a inferência do modelo
    results = model(frame)[0]

    # Extrai as detecções do resultado
    detections = sv.Detections.from_ultralytics(results)

    ### =============================================
    ### -----------> Fim do seu código <-------------
    ### =============================================

    return detections

detections = get_detections_from_frame(None)

assert isinstance(detections, sv.Detections), f"❌ Erro: O objeto retornado deveria ser do tipo 'sv.Detections', mas é {type(detections)}."
print("✅ 1/3: O objeto é do tipo correto (sv.Detections).")

assert detections.xyxy.ndim == 2, f"❌ Erro: 'xyxy' deveria ter 2 dimensões, mas tem {detections.xyxy.ndim}."
assert detections.xyxy.shape[1] == 4, f"❌ Erro: A segunda dimensão de 'xyxy' deveria ser 4 (x1, y1, x2, y2), mas é {detections.xyxy.shape[1]}."
print("✅ 2/3: O formato do array 'xyxy' está correto.")

# 4. Verifica se os valores de confiança estão no intervalo válido [0, 1]
assert np.all((detections.confidence >= 0) & (detections.confidence <= 1)), f"❌ Erro: Pelo menos um valor de confiança está fora do intervalo válido [0, 1]."
print("✅ 3/3: Os valores de confiança estão no intervalo correto [0, 1].")

print("✅ Sanity check aprovado!")

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.12/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.12/dist-packages/ultralytics/assets/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 95.6ms
image 2/2 /usr/local/lib/python3.12/dist-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 1 tie, 65.2ms
Speed: 7.7ms preprocess, 80.4ms inference, 7.2ms postprocess per image at shape (1, 3, 384, 640)
✅ 1/3: O objeto é do tipo correto (sv.Detections).
✅ 2/3: O formato do array 'xyxy' está correto.
✅ 3/3: Os valores de confiança estão no intervalo correto [0, 1].
✅ Sanity check aprovado!


In [7]:
def filter_detections_by_class(detections: sv.Detections, class_id: int = 0) -> sv.Detections:
    """Filtra um objeto Detections para manter apenas os itens da classe 'person'."""
    ### =============================================
    ### --------> Inicie seu código aqui <-----------
    ### =============================================
    mask = detections.class_id == class_id
    filtered_detections = detections[mask]

    ### =============================================
    ### -----------> Fim do seu código <-------------
    ### =============================================

    return filtered_detections

filtered_detections = filter_detections_by_class(mock_detections, class_id=0)
assert len(filtered_detections) == 2, "❌ Sanity Check Falhou: A filtragem deveria retornar 2 detecções."
print("✅ Sanity check para `filter_detections_by_class` aprovado.")

✅ Sanity check para `filter_detections_by_class` aprovado.


In [8]:
def generate_labels_for_detections(detections: sv.Detections) -> list[str]:
    """Cria uma lista de labels formatados (ex: 'person 0.85') a partir das detecções."""
    ### =============================================
    ### --------> Inicie seu código aqui <-----------
    ### =============================================

    detection_labels = [
    f"{model.names[class_id]} {confidence:.2f}"
    for class_id, confidence
    in zip(detections.class_id, detections.confidence)
    ]

    ### =============================================
    ### -----------> Fim do seu código <-------------
    ### =============================================

    return detection_labels

generated_labels = generate_labels_for_detections(filtered_detections)
assert len(generated_labels) == 2, "❌ Sanity Check Falhou: O número de labels não corresponde ao de detecções."
assert generated_labels[0] == "person 0.90", "❌ Sanity Check Falhou: O formato do label está incorreto."
print("✅ Sanity check para `generate_labels_for_detections` aprovado.")

✅ Sanity check para `generate_labels_for_detections` aprovado.


In [9]:
def annotate_frame_with_info(frame: np.ndarray, detections: sv.Detections, labels: list[str]) -> np.ndarray:
    """Aplica todas as anotações visuais (caixas, labels, contagem) em um frame."""
    annotated_frame = frame.copy()

    ### =============================================
    ### --------> Inicie seu código aqui <-----------
    ### =============================================

    # Anota o frame com as caixas e os labels
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # Adiciona o texto da contagem de pessoas no canto da tela
    person_count = len(detections)
    text = f"Pessoas: {person_count}"
    sv.draw_text(
    scene=annotated_frame,
      text=text,
      text_anchor=sv.Point(x=40, y=40),
      text_scale=2,
      text_thickness=4,
      background_color=sv.Color.BLACK,
      text_color=sv.Color.WHITE
    )

    ### =============================================
    ### -----------> Fim do seu código <-------------
    ### =============================================

    return annotated_frame

# --- Testando a função de anotação ---
dummy_frame = np.zeros((100, 100, 3), dtype=np.uint8)
annotated_dummy_frame = annotate_frame_with_info(dummy_frame, filtered_detections, generated_labels)
assert annotated_dummy_frame.shape == dummy_frame.shape, "❌ Sanity Check Falhou: As dimensões do frame mudaram."
assert not np.array_equal(annotated_dummy_frame, dummy_frame), "❌ Sanity Check Falhou: Nenhuma anotação foi desenhada no frame."
print("✅ Sanity check para `annotate_frame_with_info` aprovado.")

✅ Sanity check para `annotate_frame_with_info` aprovado.


In [10]:
def process_video(source_path: str, output_path: str):
    print("Iniciando o processamento do vídeo...")

    ### =============================================
    ### --------> Inicie seu código aqui <-----------
    ### =============================================

    # Define o gerador de frames
    frame_generator = sv.get_video_frames_generator(source_path=source_path)

    with sv.VideoSink(target_path=output_path, video_info=video_info) as sink:
        for frame in frame_generator:
            # Obter detecções
            all_detections = get_detections_from_frame(frame)

            # Filtrar detecções
            person_detections = filter_detections_by_class(all_detections)

            # Gerar labels
            labels = generate_labels_for_detections(person_detections)
            # Anotar o frame
            annotated_frame = annotate_frame_with_info(frame, person_detections, labels)

            # Gravar o frame processado no vídeo
            sink.write_frame(frame=annotated_frame)

    print(f"Processamento concluído. O vídeo foi salvo em: {output_path}")


process_video(source_path=VIDEO_PATH, output_path=OUTPUT_VIDEO_PATH)
assert os.path.exists(OUTPUT_VIDEO_PATH), f"❌ Erro: O vídeo de saída não foi encontrado em '{OUTPUT_VIDEO_PATH}'"
print(f"✅ 1/2: Arquivo de vídeo de saída '{OUTPUT_VIDEO_PATH}' foi criado com sucesso.")

output_file_size = os.path.getsize(OUTPUT_VIDEO_PATH)

assert output_file_size > 1000000, f"❌ Erro: O arquivo de vídeo de saída parece estar corrompido ou vazio (tamanho: {output_file_size / 1e6:.2f} MB)."
print(f"✅ 2/2: O arquivo de vídeo de saída não está vazio (Tamanho: {output_file_size / 1e6:.2f} MB).")
print("✅ Sanity check aprovado!")

Iniciando o processamento do vídeo...

0: 640x384 1 person, 43.9ms
Speed: 2.9ms preprocess, 43.9ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 6.0ms
Speed: 3.6ms preprocess, 6.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 chair, 7.2ms
Speed: 3.0ms preprocess, 7.2ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 2 chairs, 6.2ms
Speed: 3.7ms preprocess, 6.2ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 2 persons, 1 chair, 6.2ms
Speed: 2.6ms preprocess, 6.2ms inference, 2.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 chair, 7.3ms
Speed: 2.6ms preprocess, 7.3ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 1 chair, 6.0ms
Speed: 2.4ms preprocess, 6.0ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 3 persons, 6.2ms
Speed: 2.3ms pr

- **Exibir o vídeo** com a contagem de pessoas em tempo real.

In [11]:
from base64 import b64encode
from IPython.display import HTML, display

COMPRESSED_OUTPUT_VIDEO_PATH = "compressed_output_video.mp4"

!ffmpeg -y -i {OUTPUT_VIDEO_PATH} -vcodec libx264 -crf 28 -preset fast {COMPRESSED_OUTPUT_VIDEO_PATH} -loglevel quiet

mp4 = open(COMPRESSED_OUTPUT_VIDEO_PATH, 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

display(HTML(f"""
<video width=800 controls>
      <source src="{data_url}" type="video/mp4">
</video>
"""))


## 🧪 Resumo da Tarefa do Aluno
O objetivo é construir um pipeline completo para detecção e contagem de pessoas em um vídeo. Para isso, você deverá completar o código nos locais indicados em quatro seções principais.

Suas Tarefas:
1. Carregar o Vídeo: No primeiro bloco de código, sua tarefa é definir as variáveis VIDEO_URL e VIDEO_PATH. Preencha a URL de um vídeo .mp4 e defina o nome do arquivo local onde ele será salvo para que o download possa ser feito.

2. Inicializar os Componentes: No segundo bloco, você deve inicializar os objetos essenciais para a detecção:
    
    * Obtenha as informações do vídeo (video_info).
    * Crie os anotadores box_annotator e label_annotator.
    * Carregue um modelo YOLO pré-treinado (sugestão: yolov8n.pt).

3. Implementar a Lógica de Detecção: No terceiro bloco, você deve completar a função get_detections_from_frame. Sua tarefa é usar o model carregado para analisar um frame e converter o resultado em um objeto sv.Detections.

4. Completar as Funções de Anotação: No quarto bloco, você precisa implementar a lógica para as três funções auxiliares:

    * filter_detections_by_class: Filtre as detecções para manter apenas as que correspondem à classe "person".
    * generate_labels_for_detections: Crie os textos (labels) para cada detecção.
    * annotate_frame_with_info: Use os anotadores para desenhar as caixas, os labels e a contagem total de pessoas no quadro de vídeo.

Como Validar seu Trabalho:

Após cada seção que você completar, execute a célula de "Sanity Check" correspondente. Se todos os testes passarem (✅), você pode prosseguir para a próxima tarefa. Ao final, a execução do pipeline completo irá gerar o vídeo com suas anotações.


## 🧷 Dicas e Notas

- **Performance**: Se o processamento estiver lento (especialmente com a webcam), considere usar um modelo menor, como `yolov8n.pt` (nano) ou `yolov8s.pt` (small). Modelos maiores (`m`, `l`, `x`) são mais precisos, mas exigem mais poder computacional.
- **Webcam no Colab**: O uso de webcam (`cv2.VideoCapture(0)`) não funciona diretamente no Google Colab. Para testar com a webcam, você precisará executar este notebook em um ambiente local (no seu próprio computador).
- **Robustez**: Lembre-se que o desempenho do modelo pode variar com condições de iluminação, oclusão (pessoas se sobrepondo) ou ângulos de câmera desafiadores.
- **O Poder do supervision**: Nós usamos BoxAnnotator e LabelAnnotator, mas a biblioteca supervision é extremamente poderosa. Explore a documentação e tente usar outros anotadores, como TraceAnnotator para rastrear o movimento de objetos ou HeatMapAnnotator para visualizar áreas de maior atividade no vídeo.
- **Teste com Seus Próprios Vídeos**: Não se limite ao vídeo de exemplo. Você pode subir seu próprio arquivo .mp4 para o ambiente do Colab e ajustar a variável VIDEO_PATH no início do código para testar o pipeline em diferentes cenários e desafios.